<a href="https://colab.research.google.com/github/ISQRS00/dashboard_tc4/blob/main/TECH_CHALLENGE_FASE_%24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install statsforecast

In [14]:
!pip install prophet

In [15]:
import pandas as pd  # Utilizada para manipulação e análise de dados, especialmente em estruturas de DataFrame.
import numpy as np  # Usada para operações numéricas eficientes em arrays e matrizes.
from statsmodels.tsa.seasonal import seasonal_decompose  # Ferramenta para a decomposição sazonal de séries temporais.
import yfinance as yf  # Utilizada para baixar dados financeiros históricos diretamente do Yahoo Finance.
from statsforecast import StatsForecast  # Biblioteca para previsão de séries temporais que inclui diversos modelos.
from statsforecast.models import AutoARIMA	, SeasonalWindowAverage, AutoARIMA  # Modelos específicos para previsão de séries temporais.
import plotly.graph_objects as go  # Ferramenta para criar gráficos interativos com Plotly.
from plotly.subplots import make_subplots  # Ferramenta para criar subplots com Plotly.
from prophet import Prophet  # Biblioteca de previsão de séries temporais desenvolvida pelo Facebook.
import statsmodels.api as sm  # Ferramentas para a estimação e inferência de modelos estatísticos.
from sklearn.metrics import accuracy_score, r2_score, mean_absolute_error, mean_squared_error  # Funções para avaliar a precisão de modelos preditivos.

In [16]:
df_barril_petroleo = pd.read_csv('https://raw.githubusercontent.com/ISQRS00/dashboard_tc4/main/barril.csv', sep=';')
df_barril_petroleo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13692 entries, 0 to 13691
Data columns (total 3 columns):
 #   Column                                                                                                   Non-Null Count  Dtype  
---  ------                                                                                                   --------------  -----  
 0   Data                                                                                                     13692 non-null  object 
 1   Preço - petróleo bruto - Brent (FOB) - US$ - Energy Information Administration (EIA) - EIA366_PBRENT366  11298 non-null  object 
 2   Unnamed: 2                                                                                               0 non-null      float64
dtypes: float64(1), object(2)
memory usage: 321.0+ KB


In [17]:
df_barril_petroleo.drop(columns=['Unnamed: 2'], inplace=True)

In [18]:
df_barril_petroleo.rename(columns={'Data':'ds','Preço - petróleo bruto - Brent (FOB) - US$ - Energy Information Administration (EIA) - EIA366_PBRENT366': 'y'}, inplace=True)

In [19]:
df_barril_petroleo

,ds,y
0,20/05/1987,"18,63"
1,21/05/1987,"18,45"
2,22/05/1987,"18,55"
3,23/05/1987,NaN
4,24/05/1987,NaN
...,...,...
13687,08/11/2024,"74,04"
13688,09/11/2024,NaN
13689,10/11/2024,NaN
13690,11/11/2024,"72,19"


In [20]:
df_barril_petroleo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13692 entries, 0 to 13691
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ds      13692 non-null  object
 1   y       11298 non-null  object
dtypes: object(2)
memory usage: 214.1+ KB


In [21]:
df_barril_petroleo['ds'] = pd.to_datetime(df_barril_petroleo['ds'])
df_barril_petroleo['y'] = df_barril_petroleo['y'].str.replace(',', '.').astype(float)

<ipython-input-21-31dd0b606f9d>:1: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_barril_petroleo['ds'] = pd.to_datetime(df_barril_petroleo['ds'])


In [22]:
def wmape(y_true, y_pred):
    return np.abs(y_true - y_pred).sum() / np.abs(y_true).sum()

In [23]:
df_barril_petroleo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13692 entries, 0 to 13691
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   ds      13692 non-null  datetime64[ns]
 1   y       11298 non-null  float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 214.1 KB


In [24]:
df_barril_petroleo['y'] = df_barril_petroleo['y'].ffill()

In [25]:
df_barril_petroleo_visualizacao = df_barril_petroleo.query('ds >= "2024-01-01"')

In [26]:
# Decompor a série temporal
seasonplot = seasonal_decompose(df_barril_petroleo_visualizacao.set_index('ds')['y'], model='multiplicative', period=2)

# Criar a figura com subplots
fig = make_subplots(
    rows=4, cols=1, shared_xaxes=True,
    subplot_titles=['Série Real', 'Tendência', 'Sazonalidade', 'Resíduos'],
    vertical_spacing=0.1  # Aumentar o espaçamento vertical entre os subplots
)

# Adicionar a série real
fig.add_trace(go.Scatter(
    x=seasonplot.observed.index, y=seasonplot.observed, mode='lines', name='Série Real',
    hovertext=seasonplot.observed.index.strftime('%Y-%m-%d'), hoverinfo='text+y'
), row=1, col=1)

# Adicionar a tendência
fig.add_trace(go.Scatter(
    x=seasonplot.trend.index, y=seasonplot.trend, mode='lines', name='Tendência',
    hovertext=seasonplot.trend.index.strftime('%Y-%m-%d'), hoverinfo='text+y'
), row=2, col=1)

# Adicionar a sazonalidade
fig.add_trace(go.Scatter(
    x=seasonplot.seasonal.index, y=seasonplot.seasonal, mode='lines', name='Sazonalidade',
    hovertext=seasonplot.seasonal.index.strftime('%Y-%m-%d'), hoverinfo='text+y'
), row=3, col=1)

# Adicionar os resíduos
fig.add_trace(go.Scatter(
    x=seasonplot.resid.index, y=seasonplot.resid, mode='lines', name='Resíduos',
    hovertext=seasonplot.resid.index.strftime('%Y-%m-%d'), hoverinfo='text+y'
), row=4, col=1)

# Atualizar o layout do gráfico
fig.update_layout(
    title={
        'text': 'Decomposição Sazonal da Série Temporal',
        'y': 0.95,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    height=1000,  # Aumentar a altura total do gráfico
    width=2000,
    showlegend=False,
    margin=dict(t=150)  # Aumentar o espaço no topo para o título
)

# Atualizar títulos dos eixos
#fig.update_xaxes(title_text='Data', row=4, col=1)
fig.update_yaxes(title_text='Série Real', row=1, col=1)
fig.update_yaxes(title_text='Tendência', row=2, col=1)
fig.update_yaxes(title_text='Sazonalidade', row=3, col=1)
fig.update_yaxes(title_text='Resíduos', row=4, col=1)

# Ajustar os ticks do eixo x para mostrar todos os meses
fig.update_xaxes(
    dtick="M12",  # Define a frequência dos ticks para 1 mês
    tickformat="%Y",  # Formata os ticks para mostrar apenas o mês
    tickangle=0,  # Rotaciona os rótulos do eixo x para melhor legibilidade
    tickmode="linear",
    # rangebreaks=[  # Remove os intervalos nos finais de semana
    #     dict(bounds=["sat", "mon"])
    # ]
)

# Mostrar o gráfico
fig.show()

In [27]:
df_barril_petroleo['unique_id']='Barril'

In [28]:
train = df_barril_petroleo.loc[df_barril_petroleo['ds']  < '2024-10-01']
valid = df_barril_petroleo.loc[(df_barril_petroleo['ds'] >= '2024-10-01')]
h = valid['ds'].nunique()
h

43

In [29]:
df_barril_petroleo

,ds,y,unique_id
0,1987-05-20,18.63,Barril
1,1987-05-21,18.45,Barril
2,1987-05-22,18.55,Barril
3,1987-05-23,18.55,Barril
4,1987-05-24,18.55,Barril
...,...,...,...
13687,2024-11-08,74.04,Barril
13688,2024-11-09,74.04,Barril
13689,2024-11-10,74.04,Barril
13690,2024-11-11,72.19,Barril


In [30]:
valid.reset_index(drop=True)

,ds,y,unique_id
0,2024-10-01,75.30,Barril
1,2024-10-02,74.86,Barril
2,2024-10-03,77.57,Barril
3,2024-10-04,79.32,Barril
4,2024-10-05,79.32,Barril
5,2024-10-06,79.32,Barril
6,2024-10-07,81.74,Barril
7,2024-10-08,78.19,Barril
8,2024-10-09,77.06,Barril
9,2024-10-10,79.45,Barril


In [31]:
sf = StatsForecast(
    models=[AutoARIMA(season_length=12)],
    freq='D',
    n_jobs=-1
)

sf.fit(train)

forecast_sn = sf.predict(h=len(valid))

forecast_sn = forecast_sn.reset_index().merge(valid, on=['ds', 'unique_id'], how='left')

forecast_sn


,index,unique_id,ds,AutoARIMA,y
0,0,Barril,2024-10-01,72.064928,75.30
1,1,Barril,2024-10-02,71.796212,74.86
2,2,Barril,2024-10-03,71.765918,77.57
3,3,Barril,2024-10-04,71.923844,79.32
4,4,Barril,2024-10-05,71.925944,79.32
5,5,Barril,2024-10-06,71.946216,79.32
6,6,Barril,2024-10-07,71.938207,81.74
7,7,Barril,2024-10-08,71.923869,78.19
8,8,Barril,2024-10-09,71.926583,77.06
9,9,Barril,2024-10-10,71.924874,79.45


In [32]:
wmape_sn = wmape(forecast_sn['y'].values, forecast_sn['AutoARIMA'].values)
MAE_sn = mean_absolute_error(forecast_sn['y'].values, forecast_sn['AutoARIMA'].values)
MSE_sn = mean_squared_error(forecast_sn['y'].values, forecast_sn['AutoARIMA'].values)
R2_sn = r2_score(forecast_sn['y'].values, forecast_sn['AutoARIMA'].values)

print(f'AutoARIMA	 WMAPE: {wmape_sn:.2%}')
print(f'AutoARIMA	 MAE: {MAE_sn:.3f}')
print(f'AutoARIMA	 MSE: {MSE_sn:.4f}')
print(f'AutoARIMA	 R2: {R2_sn:.2f}')

# Criar gráfico para AutoARIMA
fig_sn = go.Figure()

# Adiciona a linha real
fig_sn.add_trace(go.Scatter(x=forecast_sn['ds'], y=forecast_sn['y'],
                            mode='lines',
                            name='Real',
                            line=dict(width=2),
                            hovertext=forecast_sn['ds'].dt.strftime('%Y-%m-%d'),
                            hoverinfo='text+y'
                            ))


fig_sn.add_trace(go.Scatter(x=forecast_sn['ds'], y=forecast_sn['AutoARIMA'],
                            mode='lines',
                            name='AutoARIMA',
                            hovertext=forecast_sn['ds'].dt.strftime('%Y-%m-%d'),
                            hoverinfo='text+y'
                            ))

fig_sn.update_layout(
    title={
        'text': 'AutoARIMA	 Forecast',
        'y': 0.9,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    xaxis_title='dia-mês-ano',
    yaxis_title='Valor',
    xaxis=dict(
        tickformat='%d-%m-/%y',
        type='date',
        dtick="D1",
        nticks=30,
    ),
    yaxis=dict(
        title='Valor',
        tickformat='.3f'
    ),
    legend_title='Legenda',
    autosize=True
)

fig_sn.show()




AutoARIMA	 WMAPE: 4.71%
AutoARIMA	 MAE: 3.554
AutoARIMA	 MSE: 18.9183
AutoARIMA	 R2: -1.88


In [33]:
season_length = 252  # Exemplo para dados diários com sazonalidade anual

# Ajustar o modelo ETS (Exponential Smoothing)
model_ets = sm.tsa.ExponentialSmoothing(train['y'], seasonal='mul', seasonal_periods=season_length).fit()

# Fazer previsões
forecast_ets = model_ets.forecast(h)
forecast_dates = pd.date_range(start=train['ds'].iloc[-1] + pd.Timedelta(days=1), periods=len(valid), freq='D')
ets_df = pd.DataFrame({'ds': forecast_dates, 'ets': forecast_ets})
ets_df = ets_df.merge(valid, on=['ds'], how='inner')

# Calcular métricas de erro
wmape_ets = wmape(ets_df['y'].values, ets_df['ets'].values)
MAE_ets = mean_absolute_error(ets_df['y'].values, ets_df['ets'].values)
MSE_ets = mean_squared_error(ets_df['y'].values, ets_df['ets'].values)
R2_ets = r2_score(ets_df['y'].values, ets_df['ets'].values)
print(f'ETS WMAPE: {wmape_ets:.2%}')
print(f'ETS MAE: {MAE_ets:.3f}')
print(f'ETS MSE: {MSE_ets:.4f}')
print(f'ETS R2: {R2_ets:.2f}')

# Visualizar as previsões com plotly
fig_ets = go.Figure()

# Adicionar a linha realizada
fig_ets.add_trace(go.Scatter(x=valid['ds'], y=valid['y'],
                             mode='lines', name='Realizado',
                             line=dict(width=2),
                             hovertext=valid['ds'].dt.strftime('%Y-%m-%d'),
                             hoverinfo='text+y'))

# Adicionar a linha da previsão ETS
fig_ets.add_trace(go.Scatter(x=ets_df['ds'], y=ets_df['ets'],
                             mode='lines', name='Forecast',
                             line=dict(width=2),
                             hovertext=ets_df['ds'].dt.strftime('%Y-%m-%d'),
                             hoverinfo='text+y'))

fig_ets.update_layout(
    title={
        'text': 'ETS Forecast',
        'y': 0.9,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    xaxis_title='dia-mês',
    yaxis_title='Valor',
    xaxis=dict(
        tickformat='%d-%m',
        type='date',
        dtick="D1",
        nticks=30,
    ),
    yaxis=dict(
        title='Valor',
        tickformat='.3f'  # Formata os ticks do eixo y para 3 casas decimais
    ),
    legend_title='Legenda',
    autosize=True
)

fig_ets.show()

ETS WMAPE: 3.51%
ETS MAE: 2.651
ETS MSE: 12.7823
ETS R2: -0.94


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.



In [34]:
m = Prophet(yearly_seasonality=True,seasonality_mode='multiplicative')
m.fit(train)

future = m.make_future_dataframe(periods=len(valid), freq='D')
prophet_df = m.predict(future)


prophet_df = prophet_df[['ds', 'yhat']].merge(valid, on='ds', how='inner')

# Calcular métricas de erro
wmape_pr = wmape(prophet_df['y'].values, prophet_df['yhat'].values)
MAE_pr = mean_absolute_error(prophet_df['y'].values, prophet_df['yhat'].values)
MSE_pr = mean_squared_error(prophet_df['y'].values, prophet_df['yhat'].values)
R2_pr = r2_score(prophet_df['y'].values, prophet_df['yhat'].values)
print(f'Prophet WMAPE: {wmape_pr:.2%}')
print(f'Prophet MAE: {MAE_pr:.3f}')
print(f'Prophet MSE: {MSE_pr:.4f}')
print(f'Prophet R2: {R2_pr:.2f}')

# Visualizar as previsões com plotly
fig_pr = go.Figure()


# Adicionar a linha realizada
fig_pr.add_trace(go.Scatter(x=valid['ds'], y=valid['y'],
                            mode='lines', name='Realizado',
                            line=dict(width=2),
                            hovertext=valid['ds'].dt.strftime('%Y-%m-%d'),
                            hoverinfo='text+y'))

# Adicionar a linha da previsão Prophet
fig_pr.add_trace(go.Scatter(x=prophet_df['ds'], y=prophet_df['yhat'],
                            mode='lines', name='Previsão',
                            line=dict(width=2),
                            hovertext=prophet_df['ds'].dt.strftime('%Y-%m-%d'),
                            hoverinfo='text+y'))

fig_pr.update_layout(
    title={
        'text': 'Previsão Prophet',
        'y': 0.9,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    xaxis_title='dia-mês',
    yaxis_title='Valor',
    xaxis=dict(
        tickformat='%d-%m',
        type='date',
        dtick="D1",
        nticks=30,
    ),
    yaxis=dict(
        title='Valor',
        tickformat='.3f'  # Formata os ticks do eixo y para 3 casas decimais
    ),
    legend_title='Legenda',
    autosize=True
)

fig_pr.show()

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpl63qjsys/1ddavtc7.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpl63qjsys/jrt5s5ri.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=60842', 'data', 'file=/tmp/tmpl63qjsys/1ddavtc7.json', 'init=/tmp/tmpl63qjsys/jrt5s5ri.json', 'output', 'file=/tmp/tmpl63qjsys/prophet_modelwl_yuxco/prophet_model-20241202143804.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
14:38:04 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
14:38:16 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


Prophet WMAPE: 19.23%
Prophet MAE: 14.509
Prophet MSE: 216.1081
Prophet R2: -31.88


In [35]:
import joblib

joblib.dump(model_ets, 'ets_model.joblib')

['ets_model.joblib']